In [39]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [51]:
import numpy as np
import torch # deep neuralnet by facebook / tensorflow by google
import torch.nn as nn
from sklearn.metrics import accuracy_score

# 데이터 읽기 함수
def load_dataset(file):
  data = np.loadtxt(file)
  print("DATA=",data)
  
  input_features = data[:,0:-1]
  print("X=",input_features)
  
  labels = np.reshape(data[:,-1],(4,1)) # 행렬 transform
  print("Y=",labels)
 
  input_features = torch.tensor(input_features, dtype=torch.float)
  labels = torch.tensor(labels, dtype=torch.float)

  return (input_features, labels)

# 모델 평가 결과 계산을 위해 텐서를 리스트로 변환하는 함수
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

In [52]:
x, y = load_dataset("/gdrive/MyDrive/machine learning/train.txt")

# layer 1 가중치 초기화
w1 = torch.randn(2,2,requires_grad=True) # 정규 분포의 random으로 초기화함
b1 = torch.randn(2,requires_grad=True) # gradient를 이용해서 update하는 것을 허용함
print("\n[Init]\nw1 = {0}".format(tensor2list(w1)))
print("b1 = {0}".format(tensor2list(b1)))

# layer 2 가중치 초기화
w2 = torch.randn(2,1,requires_grad=True)
b2 = torch.randn(1,requires_grad=True)
print("w2 = {0}".format(tensor2list(w2)))
print("b2 = {0}\n".format(tensor2list(b2)))

# Activation 함수 설정
sigmoid = nn.Sigmoid()

# 이진분류 크로스엔트로피 비용 함수 설정 
loss_func = torch.nn.BCELoss()

# 옵티마이저 함수 (역전파 알고리즘을 수행할 함수)
optimizer = torch.optim.SGD([w1,b1,w2,b2],lr=0.2) 

DATA= [[0. 0. 0.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]]
X= [[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
Y= [[0.]
 [1.]
 [1.]
 [0.]]

[Init]
w1 = [[-0.5098292231559753, -0.24761919677257538], [-1.4338699579238892, -0.13542887568473816]]
b1 = [1.0786831378936768, -0.5388879776000977]
w2 = [[-0.6118589639663696], [-0.933318018913269]]
b2 = [0.7652938961982727]



In [53]:
# 모델 학습
for epoch in range(1001):

    # H(X) 계산: forward 연산
    L2 = sigmoid(torch.add(torch.matmul(x,w1),b1)) # matrix multiplier
    hx = sigmoid(torch.add(torch.matmul(L2, w2),b2))

    # WEIGHT UPDATE
    # 1) 비용 계산
    cost = loss_func(hx, y) # BCE loss function
    # 2) 역전파 수행
    cost.backward()
    optimizer.step()

    # 100 에폭마다 비용 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())

print("\n[Learned]\nw1 = {0}".format(tensor2list(w1)))
print("b1 = {0}".format(tensor2list(b1)))
print("w2 = {0}".format(tensor2list(w2)))
print("b2 = {0}\n".format(tensor2list(b2)))

0 0.6980982422828674
100 0.5265671610832214
200 0.0001076998159987852
300 0.0
400 0.0
500 1.6391282997574308e-07
600 2.9802322387695312e-08
700 0.0
800 0.0
900 0.0
1000 0.0

[Learned]
w1 = [[-148.44139099121094, 86.83003234863281], [-152.4349822998047, 98.60673522949219]]
b1 = [3.943477153778076, -184.54684448242188]
w2 = [[-173.05380249023438], [-294.73553466796875]]
b2 = [91.56657409667969]



In [54]:
# 모델 평가
# H(X) 계산: forward 연산
L2 = sigmoid(torch.add(torch.matmul(x, w1),b1))
hx = sigmoid(torch.add(torch.matmul(L2, w2),b2))

# sigmoid 0.5 이상 -> 1, 0.5 미만 -> 0
logits = (hx > 0.5).float() # true / false -> (1 - p) / p 
predicts = tensor2list(logits)
golds = tensor2list(y)

print("\nPRED=",predicts)
print("GOLD=",golds)
print("Accuracy : {0:f}".format(accuracy_score(golds, predicts)))


PRED= [[0.0], [1.0], [1.0], [0.0]]
GOLD= [[0.0], [1.0], [1.0], [0.0]]
Accuracy : 1.000000
